In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_validate

In [2]:
import sys
!{sys.executable} -m pip install xgboost


In [3]:
df=pd.read_csv("aqardata_2.csv")

In [4]:


df.info()
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2951 entries, 0 to 2950
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mainlocation  2951 non-null   object 
 1   sublocation   2276 non-null   object 
 2   neighborhood  2951 non-null   object 
 3   frontage      2951 non-null   object 
 4   purpose       2604 non-null   object 
 5   streetwidth   2948 non-null   float64
 6   size          2951 non-null   int64  
 7   Pricepm       2951 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 184.6+ KB


,mainlocation,sublocation,neighborhood,frontage,purpose,streetwidth,size,Pricepm
0,الرياض,غرب الرياض,حي ظهرة لبن,شمال,سكني,20.0,727,1800
1,بريدة,NaN,حي مشعل,غرب,NaN,15.0,450,950
2,الخبر,NaN,حي الحمرا,غرب,تجاري,100.0,1450,3500
3,الخبر,NaN,حي الحزام الاخضر,شرق,NaN,15.0,440,2700
4,بريدة,NaN,حي الرحاب,جنوب غربي,NaN,40.0,784,950


In [5]:
df.drop(labels="sublocation",axis=1,inplace=True)
df['streetwidth'] = df['streetwidth'].fillna(int(np.mean(df["streetwidth"])))
df['purpose'].fillna('سكني', inplace = True)
outprice = df[(df['Pricepm']>= 10000)].index
df.drop(outprice,inplace = True)
df = pd.get_dummies(data=df, columns=['mainlocation', 'purpose', 'frontage', 'neighborhood'])


In [6]:
X= df.drop('Pricepm', axis = 1)
y=df["Pricepm"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [8]:
import xgboost as xgb
regressor = xgb.XGBRegressor(n_estimators = 3000,
                         learning_rate = 0.01,
                         max_depth = 11)

In [9]:
regressor.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=11, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=3000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [10]:
y_pred = regressor.predict(X_test)

In [19]:
MAE = mean_absolute_error(y_test, y_pred)
MAE

372.18442776174726

In [20]:
kf = KFold(n_splits = 5, shuffle = True, random_state =22)

In [13]:
cv = cross_validate(regressor, X, y, cv= kf, scoring = ['neg_mean_absolute_error', 'r2', 'neg_root_mean_squared_error'])
print(cv)

{'fit_time': array([23.14180398, 23.94356084, 26.79513383, 25.40402436, 24.09534979]), 'score_time': array([0.02671576, 0.03889585, 0.02892256, 0.0289259 , 0.03490591]), 'test_neg_mean_absolute_error': array([-374.53617124, -409.91532826, -411.60626035, -400.17219489,
       -348.92391999]), 'test_r2': array([0.82352527, 0.76147442, 0.74821157, 0.79670706, 0.82022539]), 'test_neg_root_mean_squared_error': array([-660.23440818, -757.34779514, -748.3036492 , -660.87394815,
       -607.3701867 ])}


In [18]:

print("Root Mean Square Error (RMSE):",-1 * cv['test_neg_root_mean_squared_error'].mean())
print("Mean Absolute Error (MAE):    ",-1 * cv['test_neg_mean_absolute_error'].mean())
print("R2:                           ",cv['test_r2'].mean())

Root Mean Square Error (RMSE): 686.8259974747324
Mean Absolute Error (MAE):     389.03077494747276
R2:                            0.7900287426587159
